In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import os
# Hyperparameters
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.01
RANDOM_SEED = 42
NUM_CLASSES = 7
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [39]:
# Transforms
transform_train = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Datasets
train_val_dataset = datasets.ImageFolder(root="./train", transform=transform_train)
test_dataset = datasets.ImageFolder(root="./test", transform=transform_test)

# Split into train and val
train_size = int(0.75 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(
    train_val_dataset,
    [train_size, val_size],
    generator=torch.Generator().manual_seed(RANDOM_SEED)
)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Compute class weights
labels = [sample[1] for sample in train_val_dataset.samples]
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)


In [40]:
# CNN Model
class CNNModel(nn.Module):
    def __init__(self, output_size=NUM_CLASSES):
        super(CNNModel, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, output_size)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
model = CNNModel().to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# Training Loop
best_val_loss = float('inf')
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0
    for images, labels in train_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Epoch [{epoch+1}/{EPOCHS}] | Train Loss: {train_loss*100:.4f} %, Acc: {train_acc*100:.4f} % | Val Loss: {val_loss*100:.4f} %, Acc: {val_acc*100:.4f} %")

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model3.pth")
        print("✅ Saved best model")

    scheduler.step()

# Evaluate on Test Set
model.load_state_dict(torch.load("best_model3.pth"))
model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"\n🎯 Final Test Accuracy: {test_acc * 100:.2f}%")


Epoch [1/50] | Train Loss: 1.9537, Acc: 0.1899 | Val Loss: 1.8473, Acc: 0.2825
✅ Saved best model
Epoch [2/50] | Train Loss: 1.8422, Acc: 0.2509 | Val Loss: 1.7872, Acc: 0.3068
✅ Saved best model
Epoch [3/50] | Train Loss: 1.8140, Acc: 0.2665 | Val Loss: 1.7610, Acc: 0.3243
✅ Saved best model
Epoch [4/50] | Train Loss: 1.7839, Acc: 0.2813 | Val Loss: 1.7862, Acc: 0.2738
Epoch [5/50] | Train Loss: 1.7745, Acc: 0.2786 | Val Loss: 1.7110, Acc: 0.3398
✅ Saved best model
Epoch [6/50] | Train Loss: 1.7624, Acc: 0.2878 | Val Loss: 1.6995, Acc: 0.3553
✅ Saved best model
Epoch [7/50] | Train Loss: 1.7416, Acc: 0.2826 | Val Loss: 1.6605, Acc: 0.3596
✅ Saved best model
Epoch [8/50] | Train Loss: 1.7480, Acc: 0.3056 | Val Loss: 1.6726, Acc: 0.3547
Epoch [9/50] | Train Loss: 1.7287, Acc: 0.3004 | Val Loss: 1.6327, Acc: 0.3855
✅ Saved best model
Epoch [10/50] | Train Loss: 1.7251, Acc: 0.2987 | Val Loss: 1.6722, Acc: 0.3208
Epoch [11/50] | Train Loss: 1.6954, Acc: 0.3086 | Val Loss: 1.6039, Acc: 0.3

C:\Users\mohamed ahmed\AppData\Local\Temp\ipykernel_3968\3061708708.py:56: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model2.pth"))


🎯 Final Test Accuracy: 48.17%


In [48]:
# Evaluate on Test Set
model = CNNModel().to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
model.load_state_dict(torch.load("best_model2.pth"))
model.eval()
test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        test_correct += (predicted == labels).sum().item()
        test_total += labels.size(0)

test_acc = test_correct / test_total
print(f"\n🎯 Final Test Accuracy: {test_acc * 100:.2f}%")

C:\Users\mohamed ahmed\AppData\Local\Temp\ipykernel_3968\3571839836.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model2.pth"))



🎯 Final Test Accuracy: 48.17%
